In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from utils import cifar10_load

model_name=''
detector_name=''

(x_train, y_train), (x_dev, y_dev), (x_test, y_test) = cifar10_load()

In [ ]:
model = keras.models.load_model(model_name)
detector = keras.models.load_model(detector_name)

In [ ]:
cw_params = {'batch_size': 1,
             'confidence': 10,
             'learning_rate': 0.1,
             'binary_search_steps': 5,
             'max_iterations': 1000,
             'abort_early': True,
             'initial_const': 0.01,
             'clip_min': 0,
             'clip_max': 1}

deepfool_params = {'nb_candidate': 10,
                   'overshoot': 0.02,
                   'max_iter': 50,
                   'clip_min': 0.,
                   'clip_max': 1.}

pgd_params = {
    
}

In [ ]:
from cleverhans.utils_keras import KerasModelWrapper
from cleverhans.attacks import CarliniWagnerL2, DeepFool, ProjectedGradientDescent, LBFGS, ElasticNetMethod
from cleverhans.utils_tf import model_eval

model_wrap = KerasModelWrapper(model)
attacks = [CarliniWagnerL2, DeepFool, ProjectedGradientDescent, LBFGS, ElasticNetMethod]

for attack, param in zip(attacks, attack_params):
    attack_wrap = attak(model_wrap)
    adv_x = attack_wrap